In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install Flask pyngrok scikit-learn


In [4]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import joblib
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from threading import Thread

app = Flask(__name__)

# Load the model
with open("/content/drive/MyDrive/model.pkl", "rb") as model_file:
    model = joblib.load(model_file)

# Load the tokenizer appropriate for your model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

@app.route('/')
def home():
    return "Sentiment Analysis API is running!"

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    text = data['text']

    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt")

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits).item()

    sentiment = "Positive" if prediction == 1 else "Negative"
    return jsonify({'prediction': sentiment})

def run():
    app.run()

# Set up the ngrok tunnel
ngrok.set_auth_token("2iMUe5DOkkFcoL7fskACcdkW235_JgMEeP8v5xVA6C5BkLQh")  # replace with your actual authtoken
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

# Start the Flask app in a separate thread
thread = Thread(target=run)
thread.start()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

 * ngrok tunnel "NgrokTunnel: "https://06f7-35-221-183-101.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [7]:
import requests
import json

# Replace with your ngrok URL
url = "https://06f7-35-221-183-101.ngrok-free.app/predict"

# Sample data
data = {"text": "I love this food!"}

# Send POST request
response = requests.post(url, json=data)

# Print the prediction result
print(response.json())

INFO:werkzeug:127.0.0.1 - - [27/Jun/2024 20:51:06] "POST /predict HTTP/1.1" 200 -


{'prediction': 'Positive'}
